In [1]:
%load_ext autoreload  
%autoreload 2 

In [2]:
import datasets
from subset_active_learning.subset_selection import select, preprocess, compare
import wandb
import numpy as np
import torch

/home/glai/anaconda3/envs/subset/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
processed_ds = preprocess.preprocess_sst2("google/electra-small-discriminator")

No config specified, defaulting to: sst/default
Reusing dataset sst (/home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)
100%|██████████| 3/3 [00:00<00:00, 519.25it/s]
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-7b23d5d1250b3cef.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-6a94e7b0eb5aeeeb.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-f1806eefe95f0de4.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-e283858a38c31b48.arrow
Loading cache

## Increase num_workers

In [8]:
INCREASE_FACTOR = 4
small_batch_config = select.SubsetTrainingArguments(batch_size=8, learning_rate=1e-5)
large_batch_config = select.SubsetTrainingArguments(batch_size=small_batch_config.batch_size*INCREASE_FACTOR, learning_rate=small_batch_config.learning_rate*(3/4))

In [ ]:
for seed in range(42, 47):
    train_ds = processed_ds["train"].shuffle(seed=seed).select(range(100))
    num_worker_comparison = compare.ComparisonRun(train_ds=train_ds, valid_ds=processed_ds["validation"], test_ds=processed_ds["test"], seed=seed)
    # num_worker is 2
    NUM_WORKERS = 2
    num_worker_comparison.one_run(wandb_tags=[f"large_batch_{large_batch_config.batch_size}", "hf_ds", f"num_workers_{NUM_WORKERS}"], config=large_batch_config, num_workers=NUM_WORKERS)
    # same as before, control group
    NUM_WORKERS = 0
    num_worker_comparison.one_run(wandb_tags=[f"large_batch_{large_batch_config.batch_size}", "hf_ds", f"num_workers_{NUM_WORKERS}"], config=large_batch_config, num_workers=NUM_WORKERS)

## Use Custom Dataloaders

In [4]:
from subset_active_learning.subset_selection import data_loader

In [5]:
# Convert HF dataset into Torch dataset for ffcv support
class InMemorySST2(torch.utils.data.Dataset):
    def __init__(self, hf_ds):
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.in_memory_ds = []
        for row in hf_ds: 
            self.in_memory_ds.append({k: v.to(device) for k, v in row.items()})
    
    def __getitem__(self, i):
        return self.in_memory_ds[i]
    
    def __len__(self):
        return len(self.in_memory_ds)

In [6]:
# train_ds = InMemorySST2(hf_ds=processed_ds["train"].shuffle(seed=seed).select(range(100)))
# train_dl = data_loader.DataLoader(dataset=train_ds, batch_size=8, shuffle=True)
# for batch in train_dl:
#     print(type(batch))
#     break
# {k: v.shape for k, v in batch.items()}

In [10]:
seed = 42
# push all data to cuda beforehand
NUM_WORKERS = 2
num_worker_comparison = compare.ComparisonRun(
    train_ds=InMemorySST2(hf_ds=processed_ds["train"].shuffle(seed=seed).select(range(100))),
    valid_ds=InMemorySST2(hf_ds=processed_ds["validation"]),
    test_ds=InMemorySST2(hf_ds=processed_ds["test"]),
    seed=seed,
)
num_worker_comparison.one_run(
    wandb_tags=[
        f"large_batch_{large_batch_config.batch_size}",
        "hf_ds",
        f"preload_data",
    ],
    config=large_batch_config,
    num_workers=NUM_WORKERS,
    use_custom_subset_trainer=True # use our custom trainer that support preload data into GPU 
)
# same as before, control group
NUM_WORKERS = 0
num_worker_comparison = compare.ComparisonRun(
    train_ds=processed_ds["train"].shuffle(seed=seed).select(range(100)),
    valid_ds=processed_ds["validation"],
    test_ds=processed_ds["test"],
    seed=seed,
)
num_worker_comparison.one_run(
    wandb_tags=[
        f"large_batch_{large_batch_config.batch_size}",
        "hf_ds",
        f"num_workers_{NUM_WORKERS}",
    ],
    config=large_batch_config,
    num_workers=NUM_WORKERS,
)

Loading cached shuffled indices for dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-6d5c67900b24328c.arrow


batch_size,▁
loss,█▄▆▅▄▃▄▄▃▄▄▃▆▅▃▅▅▇▅▄▃▃▇▆▄▆▄▅▅▃▆▄▄▃▄▆▄▂▅▁
batch_size,32
loss,0.63243


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

batch_size,▁
loss,██████▇▇▇▆▅▅▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
run_time,▁
sst2_final_test:accuracy,▁
sst2_final_valid:accuracy,▁
sst:val_acc,▁▇█▇██
batch_size,32
loss,0.00527
run_time,155.28
sst2_final_test:accuracy,0.6991
sst2_final_valid:accuracy,0.70481


Loading cached shuffled indices for dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-6d5c67900b24328c.arrow


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

batch_size,▁
loss,█████████▇▇▆▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
run_time,▁
sst2_final_test:accuracy,▁
sst2_final_valid:accuracy,▁
sst:val_acc,▁▇██▇
batch_size,32
loss,0.00921
run_time,147.94
sst2_final_test:accuracy,0.71493
sst2_final_valid:accuracy,0.71571
